<a href="https://colab.research.google.com/github/Jitmandal051004/Rice_type_Classification/blob/main/Rice_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Initialization


In [2]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/mssmartypants/rice-type-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jitmandal24
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/mssmartypants/rice-type-classification


100%|██████████| 888k/888k [00:00<00:00, 990kB/s]

In [4]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


### Pre_processing

In [6]:
data_df = pd.read_csv("/content/rice-type-classification/riceClassification.csv")
data_df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [7]:
data_df.dropna(inplace=True)
data_df.drop(['id'], axis=1, inplace=True)
print(data_df.shape)

(18185, 11)


In [8]:
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [9]:
print(data_df["Class"].unique())
print(data_df["Class"].value_counts())

[1 0]
Class
1    9985
0    8200
Name: count, dtype: int64


### Normalization

In [10]:
orig_data_df = data_df.copy()

for col in data_df.columns:
  data_df[col] = (data_df[col] / data_df[col].abs().max())

data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [11]:
X = np.array(data_df.iloc[:, :-1]) #all values except last col
Y = np.array(data_df.iloc[:, -1]) #last col

print(X.shape, Y.shape)

(18185, 10) (18185,)


### Splitting of the dataset

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

print(X_train.shape, X_test.shape, X_val.shape)

(12729, 10) (2728, 10) (2728, 10)


### Dataset object


In [21]:
class dataset(Dataset):
  def __init__(self,X,Y):
    self.X = torch.tensor(X, dtype=torch.float32).to(device) #vector
    self.Y = torch.tensor(Y, dtype=torch.float32).to(device)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    return self.X[index], self.Y[index]

In [22]:
training_data = dataset(X_train, y_train)
testing_data = dataset(X_test, y_test)
validation_data = dataset(X_val, y_val)

### Data_loader


In [23]:
train_Dataloader = DataLoader(training_data, batch_size=8, shuffle=True)
validation_data = DataLoader(validation_data, batch_size=8, shuffle=True)
testing_Dataloader = DataLoader(testing_data, batch_size=8, shuffle=True)

In [24]:
for x, y in train_Dataloader:
  print(x)
  print("========")
  print(y)
  break

tensor([[0.6477, 0.8926, 0.6354, 0.9797, 0.6150, 0.8048, 0.6635, 0.7126, 0.6994,
         0.7970],
        [0.8941, 0.8532, 0.9122, 0.9064, 0.8487, 0.9456, 0.8801, 0.7432, 0.8878,
         0.5307],
        [0.6299, 0.7844, 0.6998, 0.9471, 0.6016, 0.7936, 0.6245, 0.6556, 0.8037,
         0.6360],
        [0.8591, 0.8553, 0.8707, 0.9191, 0.8142, 0.9269, 0.8565, 0.7337, 0.8751,
         0.5573],
        [0.7318, 0.7955, 0.7984, 0.9226, 0.6922, 0.8555, 0.6888, 0.6785, 0.8719,
         0.5653],
        [0.5675, 0.7912, 0.6316, 0.9651, 0.5400, 0.7533, 0.9205, 0.6443, 0.7497,
         0.7107],
        [0.8701, 0.8692, 0.8684, 0.9236, 0.8225, 0.9328, 0.8131, 0.7488, 0.8511,
         0.5679],
        [0.3561, 0.4989, 0.6336, 0.8483, 0.3382, 0.5968, 0.8519, 0.4629, 0.9115,
         0.4467]], device='cuda:0')
tensor([1., 0., 0., 0., 0., 1., 0., 1.], device='cuda:0')


### model

In [ ]:
class MyModel(nn.Module):
  def __init__(self):

  def forward